# Advanced AI Agents with Google Calendar Integration - Lecture 2

Welcome to Lecture 2! 🎉 In this hands-on session, you'll learn how to:

1. 🤖 Build intelligent AI agents using the `openai-agents` library
2. 🔧 Create function tools that agents can use
3. 📅 Integrate AI agents with Google Calendar API
4. 🚀 Build a complete agent-powered scheduling system
5. 💬 Handle natural language processing for calendar events

## Learning Objectives

By the end of this lecture, you will be able to:
- Create sophisticated AI agents with custom tools
- Build function decorators for agent capabilities
- Integrate external APIs (Google Calendar) with AI agents
- Handle natural language input for structured data extraction
- Deploy a complete agent-powered application

## Prerequisites
- Completed Lecture 1 (Basic LLM and Calendar API setup)
- Azure OpenAI credentials configured
- Google Calendar API credentials set up
- Basic understanding of async/await in Python

Let's dive into the world of intelligent agents! 🚀

## Section 1: Setting Up Advanced Agents 🔧

In this section, we'll set up the `openai-agents` library and configure it to work with your Azure OpenAI credentials. This is more advanced than the basic LLM calls we used in Lecture 1.

### What are AI Agents?
- **Agents** are AI systems that can use tools to accomplish tasks
- **Function Tools** allow agents to call Python functions
- **Runners** execute agent workflows and handle conversations
- **Sessions** maintain memory across interactions

In [21]:
# 📦 Exercise 1.1: Import and Setup
# Install required packages first (run once)
# !pip install openai-agents google-api-python-client google-auth-httplib2 google-auth-oauthlib

print("📦 Setting up advanced AI agents...")

# Core imports
import asyncio
import os
import json
import datetime
import pickle
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Import the powerful agents library
from agents import (
    Agent,
    Runner,
    function_tool,
    set_default_openai_client,
)
from openai import AsyncAzureOpenAI

# Google Calendar libraries (from Lecture 1)
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

print("✅ All libraries imported successfully!")
print("🎯 Ready to build intelligent agents!")

📦 Setting up advanced AI agents...
✅ All libraries imported successfully!
🎯 Ready to build intelligent agents!


In [22]:
# 🔧 Exercise 1.2: Configure Azure OpenAI for Agents
print("🔧 Configuring Azure OpenAI for agents...")

# Configure Azure OpenAI client (more advanced than Lecture 1)
azure_client = AsyncAzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("OPENAI_VERSION_NAME"), 
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
)

# Set the default client for the agents library with tracing disabled
set_default_openai_client(azure_client, use_for_tracing=False)

# Completely disable tracing to avoid API key conflicts
if 'OPENAI_API_KEY' in os.environ:
    del os.environ['OPENAI_API_KEY']

print("✅ Azure OpenAI client configured for agents!")
print(f"🔗 Endpoint: {os.environ.get('AZURE_OPENAI_ENDPOINT')}")
print(f"📝 Model: {os.environ.get('OPENAI_DEPLOYMENT_NAME')}")
print("🚫 Tracing disabled to avoid conflicts")

🔧 Configuring Azure OpenAI for agents...
✅ Azure OpenAI client configured for agents!
🔗 Endpoint: https://pdpjulie-openai.openai.azure.com/
📝 Model: gpt-4o
🚫 Tracing disabled to avoid conflicts


## Section 2: Understanding Function Tools 🛠️

**Function Tools** are the secret sauce of AI agents! They allow your agent to:
- Call Python functions
- Access external APIs
- Perform calculations
- Interact with databases
- And much more!

### The `@function_tool` Decorator
This decorator converts any Python function into a tool that agents can use. The agent can:
1. **See** the function signature and docstring
2. **Understand** what the function does
3. **Call** the function with appropriate parameters
4. **Use** the function's return value in its response

In [28]:
# 🧪 Exercise 2.1: Create Your First Function Tool
print("🛠️ Creating simple function tools...")

@function_tool
async def get_current_date() -> str:
    """Get the current date in YYYY-MM-DD format"""
    return datetime.datetime.now().strftime('%Y-%m-%d')

@function_tool
async def calculate_duration(start_time: str, end_time: str) -> str:
    """
    Calculate duration between two times in HH:MM format
    
    Args:
        start_time: Start time in HH:MM format (24-hour)
        end_time: End time in HH:MM format (24-hour)
        
    Returns:
        Duration as a string (e.g., "2 hours 30 minutes")
    """
    try:
        start_hour, start_min = map(int, start_time.split(':'))
        end_hour, end_min = map(int, end_time.split(':'))
        
        start_total = start_hour * 60 + start_min
        end_total = end_hour * 60 + end_min
        
        if end_total < start_total:  # Next day
            end_total += 24 * 60
            
        duration_min = end_total - start_total
        hours = duration_min // 60
        minutes = duration_min % 60
        
        if hours > 0 and minutes > 0:
            return f"{hours} hours {minutes} minutes"
        elif hours > 0:
            return f"{hours} hours"
        else:
            return f"{minutes} minutes"
            
    except Exception as e:
        return f"Error calculating duration: {str(e)}"

@function_tool
async def get_weather_mock(city: str) -> str:
    """Get mock weather information for a city"""
    weather_data = {
        "brussels": "🌤️ Partly cloudy, 18°C",
        "london": "🌧️ Rainy, 15°C", 
        "paris": "☀️ Sunny, 22°C",
        "new york": "🌥️ Cloudy, 20°C"
    }
    return weather_data.get(city.lower(), f"🌍 Weather data not available for {city}")

🛠️ Creating simple function tools...


## 📚 AI Agent Creation Template Guide 

This section provides a comprehensive template for creating and interacting with AI agents. Use this as a reference when building your own agents!

### 🔧 Step 1: Agent Creation Template

```python
# Template for creating any AI agent
my_agent = Agent(
    name="[Agent Name]",                                    # Give your agent a clear name
    model=os.environ.get("OPENAI_DEPLOYMENT_NAME", "gpt-4o"), # Choose your AI model
    instructions="""[Agent Instructions]
    
    You are a [type of assistant] with access to [list capabilities].
    
    Your primary tasks:
    - [Task 1]: [Description]
    - [Task 2]: [Description] 
    - [Task 3]: [Description]
    
    Behavior guidelines:
    - Always be [personality trait]
    - When [situation], do [action]
    - Use tools when [condition]
    
    Communication style:
    - [Style instruction 1]
    - [Style instruction 2]""",
    tools=[                                                 # List of @function_tool functions
        function_1,
        function_2,
        function_3
    ]
)
```

### 🗣️ Step 2: How to Question Your Agent

```python
async def ask_agent_question(question: str, max_turns: int = 3):
    """
    Template function for asking questions to any agent
    
    Args:
        question: The question/request to ask the agent
        max_turns: Maximum conversation turns allowed
    
    Returns:
        Agent's response
    """
    print(f"👤 User Question: {question}")
    print("-" * 40)
    
    try:
        # Create a runner and get agent response
        result = await Runner().run(
            starting_agent=my_agent,    # Replace with your agent variable
            input=question,
            max_turns=max_turns
        )
        
        print(f"🤖 Agent Response:")
        print(f"{result.final_output}")
        return result.final_output
        
    except Exception as e:
        error_msg = f"❌ Error: {e}"
        print(error_msg)
        return error_msg
```

### 🧪 Step 3: Example Usage Pattern

```python
# Example questions to try with any agent
example_questions = [
    "What's today's date?",                                    # Simple tool usage
    "How long is a meeting from 14:30 to 16:45?",            # Calculation request
    "What's the weather like in Brussels?",                   # Information request
    "Can you help me with multiple tasks?",                   # Open-ended request
]

# Test one example question
await ask_agent_question(example_questions[0])
```

In [29]:
# 🧪 Exercise 2.2: Test Function Tools with a Simple Agent
print("🤖 Creating a simple agent to test our tools...")

# Create a simple agent with our function tools
simple_agent = Agent(
    name="Helper Assistant",
    model=os.environ.get("OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    instructions="""You are a helpful assistant with access to useful tools.
    
    Use the available tools to help users with:
    - Getting current date information
    - Calculating time durations
    - Getting weather information
    
    Always use the tools when relevant and explain what you're doing.""",
    tools=[get_current_date, calculate_duration, get_weather_mock]
)

# Test 1
print("\n1️⃣ Test: What's today's date?")
result1 = await Runner().run(
        starting_agent=simple_agent,
        input="What's today's date?",
        max_turns=2
    )
# Test 2
print("\n2️⃣ Test: How long is a meeting from 14:30 to 16:45?")
result2 = await Runner().run(
        starting_agent=simple_agent,
        input="How long is a meeting from 14:30 to 16:45?",
        max_turns=2
    )
# Test 3: Weather query
print("\n3️⃣ Test: What's the weather in Brussels?")
result3 = await Runner().run(
        starting_agent=simple_agent,
        input="What's the weather like in Brussels?",
        max_turns=2
)
print(f"📅 Agent: {result1.final_output}")
print(f"⏰ Agent: {result2.final_output}")
print(f"🌤️ Agent: {result3.final_output}")


🤖 Creating a simple agent to test our tools...

1️⃣ Test: What's today's date?

2️⃣ Test: How long is a meeting from 14:30 to 16:45?

2️⃣ Test: How long is a meeting from 14:30 to 16:45?

3️⃣ Test: What's the weather in Brussels?

3️⃣ Test: What's the weather in Brussels?
📅 Agent: Today's date is November 14, 2025. Let me know if you need assistance with something else!
⏰ Agent: The meeting from 14:30 to 16:45 is 2 hours and 15 minutes long.
🌤️ Agent: The weather in Brussels is currently partly cloudy with a temperature of 18°C. Let me know if you need more details!
📅 Agent: Today's date is November 14, 2025. Let me know if you need assistance with something else!
⏰ Agent: The meeting from 14:30 to 16:45 is 2 hours and 15 minutes long.
🌤️ Agent: The weather in Brussels is currently partly cloudy with a temperature of 18°C. Let me know if you need more details!


In [12]:
async def test_agent (): 
    # Test 1
    print("\n1️⃣ Test: What's today's date?")
    result1 = await Runner().run(
            starting_agent=simple_agent,
            input="What's today's date?",
            max_turns=2
        )
    # Test 2
    print("\n2️⃣ Test: How long is a meeting from 14:30 to 16:45?")
    result2 = await Runner().run(
            starting_agent=simple_agent,
            input="How long is a meeting from 14:30 to 16:45?",
            max_turns=2
        )
    # Test 3: Weather query
    print("\n3️⃣ Test: What's the weather in Brussels?")
    result3 = await Runner().run(
            starting_agent=simple_agent,
            input="What's the weather like in Brussels?",
            max_turns=2
    )
    print(f"📅 Agent: {result1.final_output}")
    print(f"⏰ Agent: {result2.final_output}")
    print(f"🌤️ Agent: {result3.final_output}")
    return [result1, result2, result3]
# Run the tests
results = await test_agent()



1️⃣ Test: What's today's date?

2️⃣ Test: How long is a meeting from 14:30 to 16:45?

3️⃣ Test: What's the weather in Brussels?
📅 Agent: Today's date is November 12, 2025. Let me know if you need assistance with anything else!
⏰ Agent: The meeting lasts for 2 hours and 15 minutes, from 14:30 to 16:45.
🌤️ Agent: The current weather in Brussels is partly cloudy with a temperature of 18°C.


## Section 3: Google Calendar Integration 📅

Now let's integrate Google Calendar with our agents! We'll reuse the calendar functions from Lecture 1 but turn them into powerful agent tools.

### Key Concepts:
- **Service Authentication**: Reusing your existing Google Calendar credentials
- **Function Tool Conversion**: Turning calendar functions into agent tools
- **Error Handling**: Making tools robust for agent use
- **Return Values**: Providing clear feedback to the agent

In [24]:
# 🔧 Exercise 3.1: Set up Google Calendar Service
import os
Client_id = os.getenv("GOOGLE_CLIENT_ID")
client_secret = os.getenv("GOOGLE_CLIENT_SECRET")

# If modifying these scopes, delete the file token.pickle.

SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_calendar_service():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_config(
                {
                    "installed": {
                        "client_id": Client_id,
                        "client_secret": client_secret,
                        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                        "token_uri": "https://oauth2.googleapis.com/token"
                    }
                },
                SCOPES
            )
            creds = flow.run_local_server(port=0)  # automatically opens browser and handles redirect

        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)
    return service

In [25]:
# 🛠️ Exercise 3.2: Create Calendar Function Tools
print("🛠️ Creating powerful calendar function tools...")

@function_tool
async def create_calendar_event(
    title: str,
    date: str,
    start_time: str,
    end_time: str,
    location: str = "",
    description: str = ""
) -> str:
    """
    Create a Google Calendar event
    
    Args:
        title: Event title/name (required)
        date: Event date in YYYY-MM-DD format (required)
        start_time: Start time in HH:MM format, 24-hour (required)
        end_time: End time in HH:MM format, 24-hour (required)
        location: Event location (optional)
        description: Event description (optional)
        
    Returns:
        Success message with event details and link
    """
    try:
        # Get calendar service
        service = get_calendar_service()
        
        # Create datetime strings (ISO format for Google Calendar)
        start_datetime = f"{date}T{start_time}:00"
        end_datetime = f"{date}T{end_time}:00"
        
        # Build the event structure
        event = {
            'summary': title,
            'location': location,
            'description': description or f"Event: {title}",
            'start': {
                'dateTime': start_datetime,
                'timeZone': 'Europe/Brussels',
            },
            'end': {
                'dateTime': end_datetime,
                'timeZone': 'Europe/Brussels',
            },
            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'popup', 'minutes': 15},
                    {'method': 'email', 'minutes': 60},
                ],
            },
        }
        
        # Create the event
        created_event = service.events().insert(calendarId='primary', body=event).execute()
        event_link = created_event.get('htmlLink', 'No link available')
        
        return f"✅ Successfully created event: '{title}' on {date} from {start_time} to {end_time}. Event link: {event_link}"
        
    except Exception as e:
        return f"❌ Failed to create event: {str(e)}"
##### ADDITIONAL TOOL #####
@function_tool
async def list_upcoming_events(max_results: int = 5) -> str:
    """
    List upcoming events from Google Calendar
    
    Args:
        max_results: Maximum number of events to return (default: 5)
        
    Returns:
        List of upcoming events with details
    """
    try:
        service = get_calendar_service()
        
        # Get current time in RFC3339 format
        now = datetime.datetime.utcnow().isoformat() + 'Z'
        
        events_result = service.events().list(
            calendarId='primary',
            timeMin=now,
            maxResults=max_results,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        
        if not events:
            return "📅 No upcoming events found."
        
        result = f"📅 Next {len(events)} upcoming events:\n"
        for i, event in enumerate(events, 1):
            start = event['start'].get('dateTime', event['start'].get('date'))
            title = event.get('summary', 'No title')
            location = event.get('location', 'No location')
            
            # Format the datetime for display
            if 'T' in start:
                event_datetime = datetime.datetime.fromisoformat(start.replace('Z', '+00:00'))
                formatted_time = event_datetime.strftime('%Y-%m-%d at %H:%M')
            else:
                formatted_time = start
                
            result += f"{i}. {title} - {formatted_time}"
            if location:
                result += f" at {location}"
            result += "\n"
            
        return result
        
    except Exception as e:
        return f"❌ Failed to list events: {str(e)}"

print("✅ Calendar function tools created!")
print("🔧 Tools available:")
print("  - create_calendar_event(): Creates events in Google Calendar")
print("  - list_upcoming_events(): Lists upcoming calendar events")

🛠️ Creating powerful calendar function tools...
✅ Calendar function tools created!
🔧 Tools available:
  - create_calendar_event(): Creates events in Google Calendar
  - list_upcoming_events(): Lists upcoming calendar events


## Section 4: Building the Calendar Scheduling Agent 🤖

Now comes the exciting part! We'll create an intelligent agent that can:
- 🧠 Understand natural language requests
- 📊 Extract structured data from free text
- 📅 Schedule events in Google Calendar
- 💬 Provide helpful feedback and confirmations

### Agent Design Principles:
1. **Clear Instructions**: Tell the agent exactly what it should do
2. **Tool Integration**: Give the agent access to the right tools
3. **Error Handling**: Guide the agent on how to handle problems
4. **User Experience**: Make interactions smooth and intuitive

In [26]:
# 🤖 Exercise 4.1: Create the Master Calendar Agent
print("🤖 Creating the intelligent Calendar Scheduling Agent...")

calendar_agent = Agent(
    name="Calendar Scheduler Pro",
    model=os.environ.get("OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    instructions="""You are an expert calendar scheduling assistant with access to Google Calendar.

Your primary job is to help users schedule events efficiently and accurately.

WHEN SCHEDULING EVENTS:
1. Extract key details: title, date, start time, end time, location, description
2. If ANY required information is missing (title, date, start_time, end_time), ask the user for clarification
3. Use ONLY the create_calendar_event function to schedule events
4. Always confirm successful scheduling with event details

FORMATTING REQUIREMENTS:
- Dates: Use YYYY-MM-DD format (e.g., 2025-10-29)
- Times: Use 24-hour format HH:MM (e.g., 14:30 for 2:30 PM)
- If user gives 12-hour format, convert to 24-hour

HELPFUL FEATURES:
- Use get_current_date() to handle relative dates like "tomorrow", "next Monday"
- Use list_upcoming_events() to check for scheduling conflicts
- Use calculate_duration() to help with time planning

COMMUNICATION STYLE:
- Be friendly and professional
- Always confirm details before creating events
- Provide clear success messages with event links
- Ask clarifying questions when information is incomplete

Remember: You have the power to actually create real calendar events!""",
    tools=[
        create_calendar_event, 
        list_upcoming_events, 
        get_current_date, 
        calculate_duration
    ]
)

🤖 Creating the intelligent Calendar Scheduling Agent...


In [36]:
# 🧪 Exercise 4.2: Test the Calendar Agent (One by One)

# Define test scenarios
test_scenarios = [
    {
        "name": "Simple Event",
        "input": "Schedule a team meeting tomorrow from 2:00 PM to 3:30 PM in Conference Room A"
    },
    {
        "name": "Detailed Event", 
        "input": "Create an event for 'Machine Learning Workshop' on 2025-11-15 from 09:00 to 12:00 at University Campus, Room 101. This is a hands-on workshop about neural networks."
    },
    {
        "name": "Check Upcoming Events",
        "input": "What are my upcoming events this week?"
    }
]

# Test All the scenarios one by one
result = await Runner().run(
        starting_agent=calendar_agent,
        input=test_scenarios[0]["input"],
        max_turns=5
        )
        
print(f"📅 Test '{test_scenarios[0]['name']}' Result: {result.final_output}")


result = await Runner().run(
        starting_agent=calendar_agent,
        input=test_scenarios[1]["input"],
        max_turns=5
        )
        
print(f"📅 Test '{test_scenarios[1]['name']}' Result: {result.final_output}")

result = await Runner().run(
        starting_agent=calendar_agent,
        input=test_scenarios[2]["input"],
        max_turns=5
        )
        
print(f"📅 Test '{test_scenarios[2]['name']}' Result: {result.final_output}")


📅 Test 'Simple Event' Result: Your team meeting has been successfully scheduled! Here are the details:

**Title:** Team Meeting  
**Date:** 2025-11-15 (tomorrow)  
**Time:** 14:00 to 15:30 (2:00 PM to 3:30 PM)  
**Location:** Conference Room A  
**Description:** Team meeting scheduled in Conference Room A  

You can view the event [here](https://www.google.com/calendar/event?eid=NDJudG1vN2pxMmwwdjM0M2FrajVvMThvam8ganVsaWV2YW5hY2tlcmUxNEBt).

Let me know if you need any further assistance! 😊
📅 Test 'Detailed Event' Result: The event has been successfully scheduled!

- **Title:** Machine Learning Workshop  
- **Date:** 2025-11-15  
- **Time:** 09:00 - 12:00  
- **Location:** University Campus, Room 101  
- **Description:** This is a hands-on workshop about neural networks.

You can view and manage the event using the following [link](https://www.google.com/calendar/event?eid=aThxNWU4c3JodDc5N2MxdjlqdjkxaGJ2aWMganVsaWV2YW5hY2tlcmUxNEBt).  

Let me know if you need further adjustments or a

C:\Users\JulieVanackere\AppData\Local\Temp\ipykernel_34908\2967942212.py:81: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'


📅 Test 'Check Upcoming Events' Result: Here are your upcoming events this week:

### Saturday, 2025-11-15
1. **Machine Learning Workshop**  
   **Time:** 09:00  
   **Location:** University Campus, Room 101  

2. **Zwemmen**  
   **Time:** 11:00  
   **Location:** —  

3. **Team Meeting**  
   **Time:** 14:00  
   **Location:** Conference Room A  

4. **Feest Paulien en Bélise**  
   **Time:** 18:00  
   **Location:** —  

### Sunday, 2025-11-16
5. **Etentje Eline Bday**  
   **Time:** 12:00  
   **Location:** —  

6. **Lunch Meeting with Sarah**  
   **Time:** 12:00  
   **Location:** Italian Restaurant Downtown  

Let me know if you need any updates or changes! 😊


## Section 5: Interactive Calendar Assistant 💬

Now let's create an interactive version where you can chat with the agent and schedule your own events!

### Interactive Features:
- **Conversational Interface**: Chat naturally with the agent
- **Real-time Scheduling**: Create actual calendar events
- **Smart Validation**: Agent asks for missing information  
- **Conflict Detection**: Check for scheduling conflicts
- **Flexible Input**: Handle various date/time formats

In [19]:
# 💬 Exercise 5.1: Interactive Calendar Assistant
print("💬 Setting up Interactive Calendar Assistant...")

async def interactive_calendar_chat():

    
    # For notebook testing, let's simulate a conversation
    # In a real interactive scenario, you'd get user input
    
    test_conversations = [
        "What are my upcoming events?",
        "Schedule a Python study session tomorrow from 7 PM to 9 PM at home"
    ]
    
    for i, user_input in enumerate(test_conversations, 1):
        print(f"👤 User: {user_input}")
        
        try:
            result = await Runner().run(
                starting_agent=calendar_agent,
                input=user_input,
                max_turns=3
            )
            
            print(f"🤖 Assistant: {result.final_output}")
            print("-" * 40)
            
        except Exception as e:
            print(f"❌ Error: {e}")
        
        print()

# Run the interactive session
print("🚀 Starting interactive calendar assistant...")
await interactive_calendar_chat()

print("✅ Interactive session complete!")
print("💡 Tip: Modify the 'test_conversations' list above to try your own requests!")

💬 Setting up Interactive Calendar Assistant...
🚀 Starting interactive calendar assistant...
💡 Example requests you can try:
   1. Schedule a dentist appointment next Tuesday at 2 PM
   2. What meetings do I have this week?
   3. Book a 1-hour coffee meeting with Sarah tomorrow at 10 AM
   4. Create a workout session for Friday evening
   5. Add a team standup every Monday at 9 AM

👤 User: What are my upcoming events?


C:\Users\JulieVanackere\AppData\Local\Temp\ipykernel_34908\2967942212.py:81: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'


🤖 Assistant: Here are your 5 upcoming events:

1. **Paulien jarig**  
   📅 Date: 2025-11-12  
   🌍 Location: Not specified  

2. **Etentje chiro**  
   📅 Date: 2025-11-13  
   🕒 Time: 18:00  
   🌍 Location: Not specified  

3. **Zwemmen**  
   📅 Date: 2025-11-13  
   🕒 Time: 20:30  
   🌍 Location: Not specified  

4. **Zwemmen**  
   📅 Date: 2025-11-15  
   🕒 Time: 11:00  
   🌍 Location: Not specified  

5. **Feest Paulien en Bélise**  
   📅 Date: 2025-11-15  
   🕒 Time: 18:00  
   🌍 Location: Not specified  

Let me know if you'd like to schedule or adjust any events!
----------------------------------------

👤 User: Schedule a Python study session tomorrow from 7 PM to 9 PM at home
🤖 Assistant: Your Python study session has been successfully scheduled!

**Details:**
- **Title:** Python Study Session  
- **Date:** 2025-11-13 (Tomorrow)  
- **Time:** 19:00 - 21:00  
- **Location:** Home  
- **Description:** A focused study session for Python programming  

You can view or manage the ev

## Section 6: Interactive Chat Interface 💬

Now let's create an interactive chat interface where you can have natural conversations with your calendar agent! This section demonstrates how to build a simple but effective chat loop.

In [ ]:
# 💬 Exercise 6.1: Interactive Chat Interface
print("💬 Creating interactive chat interface...")

async def chat_with_agent():
    
    conversation_history = []
    
    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye', 'q']:
                print("\\n🤖 Calendar Agent: Goodbye! Have a great day! 👋")
                break
            
            if not user_input:
                continue
            
            # Add user message to history
            conversation_history.append(f"User: {user_input}")
            
            # Get agent response
            print("\\n🤖 Calendar Agent: ", end="", flush=True)
            
            # Create a runner for streaming responses
            result = await Runner().run(
                starting_agent=calendar_agent,
                input=user_input,
                max_turns=3
            )
            agent_response = result.final_output
            print(agent_response)
            
            # Add agent response to history
            conversation_history.append(f"Agent: {agent_response}")
            print("\\n" + "-"*50 + "\\n")
            
        except KeyboardInterrupt:
            print("\\n\\n🤖 Calendar Agent: Chat interrupted. Goodbye! 👋")
            break
        except Exception as e:
            print(f"\\n❌ Error: {e}")
            print("Please try again or type 'quit' to exit.\\n")

# Note: The chat interface is ready to use!
# Uncomment the next line to start an interactive chat session:
await chat_with_agent()

print("✅ Interactive chat interface created!")
print("🎯 Uncomment the line above to start chatting with your agent!")
print("💡 Note: This function requires 'await' since it's async")
print("💡 Note: This function requires 'await' since it's async")

💬 Creating interactive chat interface...
\n🤖 Calendar Agent: 

C:\Users\JulieVanackere\AppData\Local\Temp\ipykernel_34908\2967942212.py:81: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'


Here are your events for next week:

1. **Zwemmen**
   - 📅 Date: Tuesday, 2025-11-18
   - ⏰ Time: 20:30
   - 📍 Location: No location specified

2. **Zwemmen**
   - 📅 Date: Thursday, 2025-11-20
   - ⏰ Time: 20:30
   - 📍 Location: No location specified

3. **Weekend Parijs**
   - 📅 Date: Friday, 2025-11-21
   - ⏰ Time: 15:00
   - 📍 Location: No location specified

4. **Zwemmen**
   - 📅 Date: Saturday, 2025-11-22
   - ⏰ Time: 11:00
   - 📍 Location: No location specified

Let me know if you need help managing or updating any of these! 😊
\n--------------------------------------------------\n
\n🤖 Calendar Agent: Goodbye! Have a great day! 👋
✅ Interactive chat interface created!
🎯 Uncomment the line above to start chatting with your agent!
💡 Note: This function requires 'await' since it's async
💡 Note: This function requires 'await' since it's async


## Section 7: Advanced Agent-to-Agent Collaboration 🤝

Welcome to the most advanced section of our lecture! Here we'll explore how AI agents can collaborate with each other to solve complex problems.

### 🎯 What is Agent-to-Agent Collaboration?

**Agent-to-Agent Collaboration** is when one AI agent uses another agent as a tool. This powerful pattern allows you to:

- **Specialize Agents**: Each agent becomes an expert in specific domains
- **Combine Capabilities**: Leverage multiple agents' strengths simultaneously  
- **Modular Design**: Build complex systems from simpler, reusable components
- **Scalable Architecture**: Add new capabilities without rebuilding everything

### 🔧 How It Works

1. **Master Agent**: The primary agent that coordinates and makes decisions
2. **Sub-Agent Tools**: Specialized agents converted into tools using `.as_tool()`
3. **Delegation**: The master agent decides when to use sub-agents
4. **Integration**: Results from sub-agents are integrated into the final response

### 🌟 Benefits of This Approach

- **🧠 Cognitive Load Distribution**: Each agent focuses on what it does best
- **🔄 Reusability**: Sub-agents can be used across multiple master agents
- **📈 Scalability**: Easy to add new specialized capabilities
- **🛠️ Maintainability**: Changes to one agent don't affect others
- **🎯 Precision**: Specialized agents often perform better than generalists

### 🚀 Real-World Applications

- **Calendar + Weather**: Schedule outdoor events considering weather forecasts
- **Travel + Calendar**: Plan trips while checking calendar availability
- **Email + Calendar**: Automatically schedule meetings from email requests
- **Data Analysis + Reporting**: Analyze data and generate formatted reports

Let's see this in action! 👇

In [ ]:
# 🤖 Exercise 7.1: Advanced Calendar Agent with Agent-to-Agent Collaboration
print("🤖 Creating an advanced calendar agent with agent collaboration capabilities...")

# Create an advanced agent that can delegate tasks to other agents
calendar_agent_to_agent = Agent(
    name="Calendar Scheduler Pro Max",
    model=os.environ.get("OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    instructions="""You are an advanced calendar scheduling assistant with access to Google Calendar and specialized sub-agents.

🎯 YOUR CORE MISSION:
You are the master coordinator for calendar management, capable of handling complex scheduling requests by leveraging specialized tools and delegating tasks to expert sub-agents when needed.

🔧 SCHEDULING WORKFLOW:
1. Parse user requests for event details: title, date, start_time, end_time, location, description
2. If information is missing, ask specific clarifying questions
3. Use the simple_agent tool for date calculations and time duration analysis
4. Create calendar events using the create_calendar_event function
5. Verify successful creation and provide confirmation with event links

📅 DATE & TIME HANDLING:
- Convert all dates to YYYY-MM-DD format (e.g., 2025-11-14)
- Convert all times to 24-hour format HH:MM (e.g., 14:30 for 2:30 PM)
- Use the simple_agent tool to resolve relative dates like "tomorrow", "next Friday", "in 3 days"
- Handle time zone considerations (default: Europe/Brussels)

🤝 COLLABORATION FEATURES:
- Delegate complex date calculations to the simple_agent tool
- Use the simple_agent for duration calculations and time arithmetic
- Combine multiple tool outputs to provide comprehensive scheduling solutions
- Cross-reference with existing events to avoid conflicts

🎨 COMMUNICATION STYLE:
- Be professional yet friendly and approachable
- Provide clear confirmations with all event details
- Explain any date/time conversions you make
- Offer helpful suggestions for scheduling optimization
- Use emojis appropriately to enhance user experience

🛡️ ERROR HANDLING:
- If calendar creation fails, explain the issue clearly
- Suggest alternative times if conflicts are detected
- Validate all inputs before attempting to create events
- Gracefully handle missing information with specific questions

🚀 ADVANCED CAPABILITIES:
- Multi-step scheduling workflows
- Smart conflict detection and resolution
- Integration with helper agents for complex calculations
- Comprehensive event management and organization

Remember: You're not just scheduling events - you're providing an intelligent, collaborative calendar management experience!""",
    tools=[
        create_calendar_event, 
        list_upcoming_events,
        simple_agent.as_tool(
            tool_name="date_calculator_time_helper",
            tool_description="Expert assistant for date calculations, relative date resolution (tomorrow, next week), time duration calculations, and temporal arithmetic. Use this for any date/time related questions or calculations."
        )
    ]
)



🤖 Creating an advanced calendar agent with agent collaboration capabilities...


In [ ]:
# 🧪 Exercise 7.2: Comprehensive Test Scenarios for Agent Collaboration
print("🧪 Testing advanced calendar agent with comprehensive scenarios...")

# Define advanced test scenarios that showcase agent-to-agent collaboration
advanced_test_scenarios = [
    {
        "name": "Relative Date with Duration Calculation",
        "input": "Schedule a 'Project Planning Session' next Tuesday from 2 PM to 5:30 PM in the main conference room. I need to know how long this meeting will be.",
        "expected_behavior": "Agent should use simple_agent to calculate next Tuesday's date and meeting duration"
    },
    {
        "name": "Complex Scheduling with Conflict Check",
        "input": "I want to schedule a 'Team Retrospective' tomorrow afternoon for 2 hours starting at 3 PM. Can you check if I have any conflicts and create the event?",
        "expected_behavior": "Agent should check existing events and calculate end time using sub-agent"
    },
    {
        "name": "Multi-Step Planning Request",
        "input": "Schedule three 30-minute interviews next week: Monday at 10 AM, Wednesday at 2 PM, and Friday at 9 AM. All in Room B-202.",
        "expected_behavior": "Agent should handle multiple events and use date calculation tools"
    },
    {
        "name": "Natural Language Time Processing",
        "input": "Book a 'Lunch Meeting with Sarah' the day after tomorrow from noon until 1:30 PM at the Italian restaurant downtown",
        "expected_behavior": "Agent should convert relative date and 12-hour time format"
    },
    {
        "name": "Smart Duration and Timing",
        "input": "I need a 90-minute workshop called 'AI Ethics Discussion' sometime next Friday morning. Can you suggest a good time and create it?",
        "expected_behavior": "Agent should suggest optimal timing and calculate end time"
    }
]

async def run_advanced_test_scenarios():
    """Run all advanced test scenarios for the calendar agent"""
    
    print("🚀 Starting comprehensive agent collaboration tests...\n")
    results = []
    
    for i, scenario in enumerate(advanced_test_scenarios, 1):
        print(f"📝 Test {i}: {scenario['name']}")
        print(f"👤 Request: {scenario['input']}")
        print(f"🎯 Expected: {scenario['expected_behavior']}")
        print("-" * 60)
        
        try:
            # Run the test scenario
            result = await Runner().run(
                starting_agent=calendar_agent_to_agent,
                input=scenario['input'],
                max_turns=5  # Allow more turns for complex interactions
            )
            
            print(f"🤖 Agent Response:")
            print(f"{result.final_output}")
            print("✅ Test completed successfully!")
            
            results.append({
                'scenario': scenario['name'],
                'success': True,
                'response': result.final_output
            })
            
        except Exception as e:
            error_msg = f"❌ Test failed: {str(e)}"
            print(error_msg)
            results.append({
                'scenario': scenario['name'],
                'success': False,
                'error': str(e)
            })
        
        print("=" * 60)
        print()
    
    # Summary
    successful_tests = sum(1 for r in results if r['success'])
    total_tests = len(results)
    
    print(f"📊 TEST SUMMARY:")
    print(f"✅ Successful tests: {successful_tests}/{total_tests}")
    print(f"❌ Failed tests: {total_tests - successful_tests}/{total_tests}")
    
    if successful_tests == total_tests:
        print("🎉 All tests passed! Your agent collaboration is working perfectly!")
    else:
        print("🔧 Some tests failed. Review the errors above to improve your agent.")
    
    return results

# Run the comprehensive test suite
print("🏃‍♂️ Executing advanced test scenarios...")
test_results = await run_advanced_test_scenarios()

print("\n💡 Pro Tip: Notice how the agent delegates tasks to the simple_agent tool!")
print("🔍 Watch for date calculations and duration computations being handled by collaboration!")

🧪 Testing advanced calendar agent with comprehensive scenarios...
🏃‍♂️ Executing advanced test scenarios...
🚀 Starting comprehensive agent collaboration tests...

📝 Test 1: Relative Date with Duration Calculation
👤 Request: Schedule a 'Project Planning Session' next Tuesday from 2 PM to 5:30 PM in the main conference room. I need to know how long this meeting will be.
🎯 Expected: Agent should use simple_agent to calculate next Tuesday's date and meeting duration
------------------------------------------------------------
🤖 Agent Response:
Your 'Project Planning Session' has been successfully scheduled! 🎉

### Event Details:
- **Title:** Project Planning Session  
- **Date:** Tuesday, November 19, 2025  
- **Time:** 2:00 PM - 5:30 PM (3 hours and 30 minutes)  
- **Location:** Main Conference Room  
- **Description:** A session dedicated to planning upcoming projects and aligning goals.  

You can access the event using [this link](https://www.google.com/calendar/event?eid=NGMxdTE2ZnZwc

C:\Users\JulieVanackere\AppData\Local\Temp\ipykernel_34908\2967942212.py:81: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'


🤖 Agent Response:
🎉 Your **Team Retrospective** has been successfully scheduled for **November 15, 2025**, from **3:00 PM to 5:00 PM**. Here's the event link for further details or edits:

👉 [Team Retrospective Event](https://www.google.com/calendar/event?eid=OGZ2cG9oNGFqYWMxYnV2bGFvb2VzbW5zdGsganVsaWV2YW5hY2tlcmUxNEBt)

Anything else you'd like to arrange or adjust? 😊
✅ Test completed successfully!

📝 Test 3: Multi-Step Planning Request
👤 Request: Schedule three 30-minute interviews next week: Monday at 10 AM, Wednesday at 2 PM, and Friday at 9 AM. All in Room B-202.
🎯 Expected: Agent should handle multiple events and use date calculation tools
------------------------------------------------------------
🤖 Agent Response:
Your interviews have been successfully scheduled! Here are the details:

1. **Monday, November 20**, from **10:00 AM to 10:30 AM** in **Room B-202**. [View Event](https://www.google.com/calendar/event?eid=ZWtvdXZ1N2ZqMGFjamVhaW90bGY0MGloazgganVsaWV2YW5hY2tlcmUxNEBt) 